# HCI Week 2 - The disk around Beta Pictoris
*Matthew Kenworthy // Leiden Observatory*


You will analyse coronagraphic data of Beta Pictoris and investigate which one of three methods gives the clearest signal for the disk.


Two images were taken - one of beta Pic and one of a nearby reference star aligned behind the coronagraph. *Data reproduced here by kind permission of Paul Kalas. Observation date 10 October 1993. PSF reference star is HR 919.*

**REMEMBER** that coordinates in `numpy` arrays are transposed, so that point $(x,y)$ is at `[y,x]`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
%matplotlib inline

import matplotlib as mpl
mpl.rc('image', interpolation='nearest', origin='lower')

def zoom(im,x,y,bb):
    '''returns a square 2D image cut out from im centered on location (x,y) and with half-width bb'''
    return(im[y-bb:y+bb,x-bb:x+bb])

def rotate2(img, angle, c_in):
    '''rotate input image img by angle degrees about a point c_in
    c_in is an np.array((y,x))
    returns the rotated image with zeroes for unknown values'''
    from scipy.ndimage.interpolation import affine_transform
    a=angle*np.pi/180.0
    transform=np.array([[np.cos(a),-np.sin(a)],[np.sin(a),np.cos(a)]])
    offset=c_in-c_in.dot(transform)
    dst=affine_transform(img,transform.T,order=2,offset=offset,output_shape=(img.shape),cval=0.0)
    return(dst)


Open the images in `ds9` and determine the location of beta Pictoris and write down the coordinates in `xcen` and `ycen`.

**Q1 (2 points):** Show that you have the right location for the star by displaying the image with a box of 40 pixels width centered on the star.



In [ ]:
# loading in the images into the Notebook.

pic1, pic1_header = fits.getdata("beta_pic_kalas_1995/pic1.fits", header=True)
ref1, ref1_header = fits.getdata("beta_pic_kalas_1995/hr9.1.fits", header=True)

In [ ]:
# visually determine the center of beta pic by examining the image in DS9

# Q1 answer here


**Q2 (4 points):** Select a sky background region of 50 by 50 pixels that has no defects or stars in it. Display the region using `imshow()`. Determine background noise using `np.mean()` and `np.std()`. Print out both values to 2 significant figures using the `print()` command and the Pythonic [formatters detailed on this page](https://www.python-course.eu/python3_formatted_output.php).

In [ ]:
# Q2 answer here


## Method 1: Unsharp masking with Top Hat Kernel

Read the `astropy` documents on [convolution](http://astropy.readthedocs.org/en/latest/convolution/) and [kernels](http://astropy.readthedocs.org/en/latest/convolution/kernels.html). 

**Q3 (5 points):** Using the `circle_mask()` routine you wrote last week, make a square image 11 pixels on a side that contains a circle 11 pixels in diameter. Normalise the total flux in the image to 1. This will be your **Top Hat smoothing kernel**. 

Note that you can also do this with `Tophat2DKernel` below. Convolve the Beta Pic image with this kernel to produce an image that only follows the low spatial frequencies. Subtract this image off the original Beta Pic image, write it out to a FITS file, and look at it in DS9.



In [ ]:
from astropy.convolution import convolve, Gaussian2DKernel, Tophat2DKernel

# Q3 answer here

filename = 'usm_tophat.fits'
hdu = fits.PrimaryHDU(usm)
hdu.writeto(filename, overwrite=True)


## Unsharp Masking with Gaussian Kernel
Now carry out the unsharp masking with the Gaussian Kernel in `Gaussian2DKernel()`. Try different values for the sigma of the convolution kernel from 5 to 0.5, writing out the unsharp masked image first and examining it with ds9. 

**Q4 (2 points):** What happens to the disk and the halo for different values of the Kernel width?

In [ ]:
# write Q4 answer and smoothing for Kernel of 1.5 here

filename = 'usm_gaussian.fits'
hdu = fits.PrimaryHDU(usm2)
hdu.writeto(filename, overwrite=True)



## Method 2: Subtracting azimuthal averages
Here we take a set of concentric rings centered on beta Pic, calculate the mean value in the ring, and subtract it off the beta Pictoris image.

In the routing `r_theta` below, we generate an image with radial distance `r` and angle from the x-axis `\phi` that we can use to make more masks.

In [ ]:

# make circular rings

def r_theta(im, xc, yc):
    '''make a radius mask'''
    # returns the radius rr and the angle phi for point (xc,yc)
    ny, nx = im.shape
    yp, xp = np.mgrid[0:ny,0:nx]
    yp = yp - yc
    xp = xp - xc
    rr = np.sqrt(np.power(yp,2.) + np.power(xp,2.))
    phi = np.arctan2(yp, xp)
    return(rr, phi)

# show what it produces with a demonstration image

im1 = np.zeros((10,20))
r,t = r_theta(im1, 10,5)

plt.imshow(r)

In [ ]:
mask = (r > 6) * (r < 9)
plt.imshow(mask)

We can now make a routine that makes several concentric rings in one mask image, and can then use this as a mask for selecting successively larger rings in each image of beta Pictoris. An example of the output is shown below. Look at the `for` loop in the routine, and look at how `enumerate()` generates index values in `i`.

**NOTE** there is a way to generate this image *without* using a `for` loop...

In [ ]:
im2 = np.zeros((40,100))

def rings(im, x, y, r_rings):
    # makes a mask for rings of different radii specified in r_rings
    # and centre (x,y)
    # invalid values are -1
    # first, second, third... ring has value (0, 1, 2, ....)
    # ring 0 is from r_rings[0] to r_rings[1]
    # ring 1 is from r_rings[1] to r_rings[2]
    im_rings = np.zeros_like(im) - 1.
    
    # make r_inner and r_outer
    r_inner = r_rings[0:-1]
    r_outer = r_rings[1:]
    
    r, t = r_theta(im_rings, x, y)
    for i, (rin, rout) in enumerate(zip(r_inner,r_outer)):
        im_rings[(r >= rin) * (r < rout)] = i


    r_middle = (r_inner + r_outer) / 2.
    return(im_rings, r_middle)

r_rings = np.array([0,10,20,30,40,50]) # 6 points, 5 bins

im_rin, r_radius = rings(im2,10,10, r_rings)
# test that rings are complete and complementary
plt.imshow(im_rin)

print (r_radius)

nbins = r_radius.size
print (nbins)

In [ ]:
r_rings = np.array([0,10,20,30,40,50]) # 6 points, 5 bins

im_rings, r_radius = rings(pic1, xcen, ycen, r_rings)

# take mean, median and subtract off from 0 to 20 arcsec

r_mean = np.zeros_like(r_radius)
r_rms  = np.zeros_like(r_radius)

# copy ring 4 to blank image

im_test = np.zeros_like(pic1)
im_test[(im_rings == 3)] = pic1[(im_rings == 3)]

plt.imshow(im_test)

** Q5 (6 points):** Make a set of rings using `np.linspace()` with radii from 0 to 200. Vary the number of rings from 10 to 100, and then subtract off the `mean()` of each ring to produce an output image that you can examine with `ds9`.

In [ ]:
# Q5 answer here

filename = 'pic_ringsubtract.fits'
hdu = fits.PrimaryHDU(bd)
hdu.writeto(filename, overwrite=True)


## Method 3: Subtract off a reference star

We load in another image of a nearby refence star with no disk around it, scale it and subtract it off. The exposure time of the reference star and its brightness may not be the same as that of beta Pictoris, so you will have to experiment to get the right scaling factor.


**Q6 (2 points):** Find the best scaling factor, ccording to your best judgement.

In [ ]:
# Q6 answer here

filename = 'pic_refsub.fits'
hdu = fits.PrimaryHDU(pic_delt)
hdu.writeto(filename, overwrite=True)

## Displaying your best image of the Beta Pic disk

Display one of the three methods you have tested today, in a box that is about 200 pixels wide and set the image colour range from -5 to +10.

Rotate the image of beta Pictoris so that the disk is horiztontal, and label the plot in units of Astronomical Units. **HINT** look at the options for `imshow()` that include `extent` and `vmin` and `vmax`.

**Q7 (3 points):** What is the greatest extent of the disk in AU?

Are both sides of the disk the same size?

At what inner radius (in AU) would you *trust* the data? **HINT** where do the PSF removal residuals dominate?


In [ ]:
# North should be up, east left, pixel scale 0.41”/pix.
pscale = 0.41 # arcsec/pixel
dist = 19.44 # parsecs

# END Q7 answers
